# Imports and General Settings

In [29]:
import pandas as pd
import numpy as np
import requests
import json
from sqlalchemy import create_engine

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Variables for API calls

In [20]:
league_ids: list = ['39', '140', '135', '78', '61']
seasons: list = ['2022', '2021', '2020', '2019', '2018', '2017'] # add new seasons values to the front
countries: list = ['England', 'Spain', 'Italy', 'Germany', 'France']

# Local PostgreSQL Connection

In [ ]:
POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres'
POSTGRES_PASSWORD = '################'
POSTGRES_DBNAME = 'cs689_term_project'

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
    username=POSTGRES_USERNAME,
    password=POSTGRES_PASSWORD,
    ipaddress=POSTGRES_ADDRESS,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DBNAME))

engine = create_engine(postgres_str)

# Get fixtures_raw Data

In [13]:
def get_fixture_data(league_ids: list, seasons: list):
    url_fixtures = "https://api-football-v1.p.rapidapi.com/v3/fixtures"
    fixtures_headers = {
        "X-RapidAPI-Key": "##########################################",
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }
    fixtures_data = pd.DataFrame()
    for league in league_ids:
        for year in seasons:
            fixtures_querystring = {"league": league, "season": year}
            fixtures_response = requests.request(
                method="GET", 
                url=url_fixtures, 
                headers=fixtures_headers, 
                params=fixtures_querystring,
            )
            fixtures_response_json = json.loads(fixtures_response.text)
            league_season_fixtures = pd.json_normalize(fixtures_response_json['response'])
            fixtures_data = pd.concat(objs=[fixtures_data, league_season_fixtures])
    return fixtures_data

fixtures_raw: pd.DataFrame = get_fixture_data(league_ids=league_ids, seasons=seasons)
fixtures_raw.to_csv(path_or_buf='fixtures_raw_new.csv', index=False)

In [21]:
# Read from .csv to avoid API calls
# Remove after testing
fixtures_raw = pd.read_csv(filepath_or_buffer='fixtures_raw_new.csv')
fixtures_raw.head()

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,fixture.status.long,fixture.status.short,fixture.status.elapsed,league.id,league.name,league.country,league.logo,league.flag,league.season,league.round,teams.home.id,teams.home.name,teams.home.logo,teams.home.winner,teams.away.id,teams.away.name,teams.away.logo,teams.away.winner,goals.home,goals.away,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away,referee
0,867946,A. Taylor,UTC,2022-08-05T19:00:00+00:00,1659726000,1.659726e+09,1.659730e+09,525.0,Selhurst Park,London,Match Finished,FT,90.0,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb.svg,2022,Regular Season - 1,52,Crystal Palace,https://media.api-sports.io/football/teams/52.png,False,42,Arsenal,https://media.api-sports.io/football/teams/42.png,True,0.0,2.0,0.0,1.0,0.0,2.0,NaN,NaN,NaN,NaN,A. Taylor
1,867947,A. Madley,UTC,2022-08-06T11:30:00+00:00,1659785400,1.659785e+09,1.659789e+09,535.0,Craven Cottage,London,Match Finished,FT,90.0,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb.svg,2022,Regular Season - 1,36,Fulham,https://media.api-sports.io/football/teams/36.png,NaN,40,Liverpool,https://media.api-sports.io/football/teams/40.png,NaN,2.0,2.0,1.0,0.0,2.0,2.0,NaN,NaN,NaN,NaN,A. Madley
2,867948,P. Bankes,UTC,2022-08-06T14:00:00+00:00,1659794400,1.659794e+09,1.659798e+09,504.0,Vitality Stadium,"Bournemouth, Dorset",Match Finished,FT,90.0,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb.svg,2022,Regular Season - 1,35,Bournemouth,https://media.api-sports.io/football/teams/35.png,True,66,Aston Villa,https://media.api-sports.io/football/teams/66.png,False,2.0,0.0,1.0,0.0,2.0,0.0,NaN,NaN,NaN,NaN,P. Bankes
3,867949,R. Jones,UTC,2022-08-06T14:00:00+00:00,1659794400,1.659794e+09,1.659798e+09,546.0,Elland Road,"Leeds, West Yorkshire",Match Finished,FT,90.0,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb.svg,2022,Regular Season - 1,63,Leeds,https://media.api-sports.io/football/teams/63.png,True,39,Wolves,https://media.api-sports.io/football/teams/39.png,False,2.0,1.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,R. Jones
4,867950,J. Gillett,UTC,2022-08-07T13:00:00+00:00,1659877200,1.659877e+09,1.659881e+09,547.0,King Power Stadium,"Leicester, Leicestershire",Match Finished,FT,90.0,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb.svg,2022,Regular Season - 1,46,Leicester,https://media.api-sports.io/football/teams/46.png,NaN,55,Brentford,https://media.api-sports.io/football/teams/55.png,NaN,2.0,2.0,1.0,0.0,2.0,2.0,NaN,NaN,NaN,NaN,J. Gillett


In [14]:
fixtures_raw['referee'] = np.where(
    fixtures_raw['fixture.referee'].str.contains(
        pat=',',
    ), 
    fixtures_raw['fixture.referee'].str.split(',').str[0].str.split(' ').str[0].str[0] + '. ' + fixtures_raw['fixture.referee'].str.split(',').str[0].str.split(' ').str[1], # if above condition is True, do this
    fixtures_raw['fixture.referee'], # if condition is False, do this
)
fixtures_raw.head()

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,fixture.status.long,fixture.status.short,fixture.status.elapsed,league.id,league.name,league.country,league.logo,league.flag,league.season,league.round,teams.home.id,teams.home.name,teams.home.logo,teams.home.winner,teams.away.id,teams.away.name,teams.away.logo,teams.away.winner,goals.home,goals.away,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away,referee
0,867946,A. Taylor,UTC,2022-08-05T19:00:00+00:00,1659726000,1.659726e+09,1.659730e+09,525,Selhurst Park,London,Match Finished,FT,90.0,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb.svg,2022,Regular Season - 1,52,Crystal Palace,https://media.api-sports.io/football/teams/52.png,False,42,Arsenal,https://media.api-sports.io/football/teams/42.png,True,0.0,2.0,0.0,1.0,0.0,2.0,None,None,None,None,A. Taylor
1,867947,A. Madley,UTC,2022-08-06T11:30:00+00:00,1659785400,1.659785e+09,1.659789e+09,535,Craven Cottage,London,Match Finished,FT,90.0,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb.svg,2022,Regular Season - 1,36,Fulham,https://media.api-sports.io/football/teams/36.png,None,40,Liverpool,https://media.api-sports.io/football/teams/40.png,None,2.0,2.0,1.0,0.0,2.0,2.0,None,None,None,None,A. Madley
2,867948,P. Bankes,UTC,2022-08-06T14:00:00+00:00,1659794400,1.659794e+09,1.659798e+09,504,Vitality Stadium,"Bournemouth, Dorset",Match Finished,FT,90.0,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb.svg,2022,Regular Season - 1,35,Bournemouth,https://media.api-sports.io/football/teams/35.png,True,66,Aston Villa,https://media.api-sports.io/football/teams/66.png,False,2.0,0.0,1.0,0.0,2.0,0.0,None,None,None,None,P. Bankes
3,867949,R. Jones,UTC,2022-08-06T14:00:00+00:00,1659794400,1.659794e+09,1.659798e+09,546,Elland Road,"Leeds, West Yorkshire",Match Finished,FT,90.0,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb.svg,2022,Regular Season - 1,63,Leeds,https://media.api-sports.io/football/teams/63.png,True,39,Wolves,https://media.api-sports.io/football/teams/39.png,False,2.0,1.0,1.0,1.0,2.0,1.0,None,None,None,None,R. Jones
4,867950,J. Gillett,UTC,2022-08-07T13:00:00+00:00,1659877200,1.659877e+09,1.659881e+09,547,King Power Stadium,"Leicester, Leicestershire",Match Finished,FT,90.0,39,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb.svg,2022,Regular Season - 1,46,Leicester,https://media.api-sports.io/football/teams/46.png,None,55,Brentford,https://media.api-sports.io/football/teams/55.png,None,2.0,2.0,1.0,0.0,2.0,2.0,None,None,None,None,J. Gillett


In [60]:
fixtures_raw.to_sql(name='fixtures_raw_new', con=engine, index=False, if_exists='replace')

968

# Get venues_raw Data

In [5]:
# Avoiding the API call here because it takes a little bit of time and costs money.
def get_venue_data(countries: list):
    url = "https://api-football-v1.p.rapidapi.com/v3/venues"
    headers = {
        "X-RapidAPI-Key": "######################################",
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }
    venues_data = pd.DataFrame()
    for country in countries:
        querystring = {"country": country}
        response = requests.request(
            method="GET", 
            url=url, 
            headers=headers, 
            params=querystring,
        )
        response_json = json.loads(response.text)
        country_venues = pd.json_normalize(response_json['response'])
        venues_data = pd.concat(objs=[venues_data, country_venues])
    return venues_data

venues_raw: pd.DataFrame = get_venue_data(countries=countries)
venues_raw.to_csv(path_or_buf='venues_raw_new.csv', index=False)

In [22]:
# Read from .csv to avoid API calls
# Remove after testing
venues_raw = pd.read_csv(filepath_or_buffer='venues_raw_new.csv')
venues_raw.head()

,id,name,address,city,country,capacity,surface,image
0,489,Wembley Stadium,"Stadium Way, Wembley, Brent",London,England,90000,grass,https://media.api-sports.io/football/venues/48...
1,556,Old Trafford,Sir Matt Busby Way,Manchester,England,76212,grass,https://media.api-sports.io/football/venues/55...
2,562,St. James' Park,St. James&apos; Street,Newcastle upon Tyne,England,52389,grass,https://media.api-sports.io/football/venues/56...
3,504,Vitality Stadium,"Dean Court, Kings Park","Bournemouth, Dorset",England,12000,grass,https://media.api-sports.io/football/venues/50...
4,535,Craven Cottage,Stevenage Road,London,England,25700,grass,https://media.api-sports.io/football/venues/53...


# -----------------------------------------------------------------------------

# dim_location_new (SCD 2)
  
Calculate dim_location with new data (in this case, with season for 2022).

In [23]:
dim_location_raw_columns: list = [
    'fixture.venue.id',
    'league.country',
    'fixture.venue.city',
    'fixture.venue.name',
    'teams.home.name',
    'fixture.date',
]
dim_location_raw: pd.DataFrame = fixtures_raw[dim_location_raw_columns]

dim_location_grouped: pd.DataFrame = dim_location_raw.groupby(by=['fixture.venue.name']).agg({'fixture.date': [np.min,np.max]}).reset_index()
dim_location_grouped.columns = dim_location_grouped.columns.get_level_values(1)
dim_location_min_max_date: pd.DataFrame = dim_location_grouped.rename(columns={'': 'venue_name', 'amin': 'start_date', 'amax': 'end_date'})

dim_location_date_raw_merge: pd.DataFrame = dim_location_min_max_date.merge(
    right=dim_location_raw.drop_duplicates(subset=['fixture.venue.name'], keep='first'), 
    left_on='venue_name', 
    right_on='fixture.venue.name', 
    how='left',
)
dim_location_date_raw_merge.head()

,venue_name,start_date,end_date,fixture.venue.id,league.country,fixture.venue.city,fixture.venue.name,teams.home.name,fixture.date
0,Abanca-Balaídos,2019-08-17T15:00:00+00:00,2023-06-04T00:00:00+00:00,1467.0,Spain,Vigo,Abanca-Balaídos,Celta Vigo,2022-08-13T15:00:00+00:00
1,Allianz Arena,2017-08-18T18:30:00+00:00,2023-05-20T00:00:00+00:00,700.0,Germany,München,Allianz Arena,Bayern Munich,2022-08-14T15:30:00+00:00
2,Allianz Arena (München),2019-05-04T13:30:00+00:00,2019-05-04T13:30:00+00:00,NaN,Germany,NaN,Allianz Arena (München),Bayern Munich,2019-05-04T13:30:00+00:00
3,Allianz Riviera,2017-08-11T17:00:00+00:00,2023-06-03T00:00:00+00:00,663.0,France,Nice,Allianz Riviera,Nice,2022-08-14T15:05:00+00:00
4,Allianz Stadium,2017-08-19T16:00:00+00:00,2023-05-28T00:00:00+00:00,909.0,Italy,Torino,Allianz Stadium,Juventus,2022-08-15T18:45:00+00:00


In [24]:
dim_location_date_raw_venues_merge: pd.DataFrame = dim_location_date_raw_merge.merge(
    right=venues_raw, how='left', left_on='fixture.venue.id', right_on='id',
)
# When a venue becomes out of date, its ID is dropped in the API.
# Using that, it is possible to filter out what venues are not current.
dim_location_date_raw_venues_merge['is_current'] = np.where(
    dim_location_date_raw_venues_merge['id'].isna(), False, True,
)
dim_location_date_raw_venues_merge['end_date'] = np.where(
    dim_location_date_raw_venues_merge['is_current'] == True, '9999-01-01T01:00:00+00:00', dim_location_date_raw_venues_merge.end_date,
)

dim_location_columns: list = [
    'league.country',
    'fixture.venue.city',
    'address',
    'venue_name',
    'teams.home.name',
    'capacity',
    'surface',
    'start_date',
    'end_date',
    'is_current',
]
dim_location_cleaned: pd.DataFrame = dim_location_date_raw_venues_merge[dim_location_columns]
dim_location_prefinal: pd.DataFrame = dim_location_cleaned.rename(
    columns={
        'league.country': 'country',
        'fixture.venue.city': 'city',
        'teams.home.name': 'team_name',
    },
)
dim_location: pd.DataFrame = dim_location_prefinal.reset_index().rename(columns={'index': 'location_key'})

# SCD type 2 requires 2 separate keys/ids.
# The first is the dimension key, which is called location_key in this design.
# The second is venue_id, which identifies the venue (unlike the location key,
# it is not unique and can be repeated)
# Since non-current venues do not have an id, team names are used to assign those
# records the same ids as their current equivalents.
team_names: np.ndarray = fixtures_raw['teams.home.name'].unique()
team_encoding: pd.DataFrame = pd.DataFrame(data={'team_name': team_names})
team_encoding['venue_id'] = team_encoding.index + 100

dim_location: pd.DataFrame = dim_location.merge(right=team_encoding, how='left', left_on='team_name', right_on='team_name')

In [25]:
dim_location.head()

,location_key,country,city,address,venue_name,team_name,capacity,surface,start_date,end_date,is_current,venue_id
0,0,Spain,Vigo,Avenida de Balaídos,Abanca-Balaídos,Celta Vigo,31800.0,grass,2019-08-17T15:00:00+00:00,9999-01-01T01:00:00+00:00,True,132
1,1,Germany,München,Werner-Heisenberg-Allee 25,Allianz Arena,Bayern Munich,75000.0,grass,2017-08-18T18:30:00+00:00,9999-01-01T01:00:00+00:00,True,197
2,2,Germany,NaN,NaN,Allianz Arena (München),Bayern Munich,NaN,NaN,2019-05-04T13:30:00+00:00,2019-05-04T13:30:00+00:00,False,197
3,3,France,Nice,Boulevard des Jardiniers,Allianz Riviera,Nice,35624.0,grass,2017-08-11T17:00:00+00:00,9999-01-01T01:00:00+00:00,True,230
4,4,Italy,Torino,Strada Comunale di Altessano 131,Allianz Stadium,Juventus,45666.0,grass,2017-08-19T16:00:00+00:00,9999-01-01T01:00:00+00:00,True,167


In [26]:
dim_location.shape

(208, 12)

In [27]:
dim_location.to_sql(name='stg_location', con=engine, index=False, if_exists='replace')

208

# dim_date_new

In [33]:
dim_date_raw_columns: list = ['fixture.date', 'fixture.timezone']
dim_date_raw: pd.DataFrame = fixtures_raw[dim_date_raw_columns]

dim_date_raw['datetime'] = pd.to_datetime(dim_date_raw['fixture.date'])
dim_date_raw['year'] = dim_date_raw['datetime'].dt.year
dim_date_raw['month'] = dim_date_raw['datetime'].dt.month
dim_date_raw['day'] = dim_date_raw['datetime'].dt.day
dim_date_raw['hour'] = dim_date_raw['datetime'].dt.hour
dim_date_raw['minute'] = dim_date_raw['datetime'].dt.minute
dim_date_raw['second'] = dim_date_raw['datetime'].dt.second
dim_date_raw['date'] = dim_date_raw['datetime'].dt.date
dim_date_raw['time'] = dim_date_raw['datetime'].dt.time

dim_date_prefinal: pd.DataFrame = dim_date_raw.drop_duplicates().reset_index(drop=True).rename(
    columns={
        'fixture.date': 'og_datetime', 
        'fixture.timezone': 'timezone',
    },
)
dim_date_prefinal['date_key'] = dim_date_prefinal.index

dim_date_columns: list = [
    'date_key',
    'og_datetime',
    'datetime',
    'year',
    'month',
    'day',
    'hour',
    'minute',
    'second',
    'date',
    'time',
    'timezone',
]
dim_date: pd.DataFrame = dim_date_prefinal[dim_date_columns]
print(dim_date.shape)
dim_date.head()

(5837, 12)


/var/folders/k9/m9q_29c95h182lx7dl04_x3w0000gn/T/ipykernel_85520/187151441.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_date_raw['datetime'] = pd.to_datetime(dim_date_raw['fixture.date'])
/var/folders/k9/m9q_29c95h182lx7dl04_x3w0000gn/T/ipykernel_85520/187151441.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_date_raw['year'] = dim_date_raw['datetime'].dt.year
/var/folders/k9/m9q_29c95h182lx7dl04_x3w0000gn/T/ipykernel_85520/187151441.py:6: SettingWithCopyWarning: 
A value is trying to be 

,date_key,og_datetime,datetime,year,month,day,hour,minute,second,date,time,timezone
0,0,2022-08-05T19:00:00+00:00,2022-08-05 19:00:00+00:00,2022,8,5,19,0,0,2022-08-05,19:00:00,UTC
1,1,2022-08-06T11:30:00+00:00,2022-08-06 11:30:00+00:00,2022,8,6,11,30,0,2022-08-06,11:30:00,UTC
2,2,2022-08-06T14:00:00+00:00,2022-08-06 14:00:00+00:00,2022,8,6,14,0,0,2022-08-06,14:00:00,UTC
3,3,2022-08-07T13:00:00+00:00,2022-08-07 13:00:00+00:00,2022,8,7,13,0,0,2022-08-07,13:00:00,UTC
4,4,2022-08-06T16:30:00+00:00,2022-08-06 16:30:00+00:00,2022,8,6,16,30,0,2022-08-06,16:30:00,UTC


In [34]:
dim_date.to_sql(name='stg_date', con=conn, index=False, if_exists='replace')

837

# dim_referee

In [39]:
dim_referee_raw_columns: list = ['referee']
dim_referee_raw: pd.DataFrame = fixtures_raw[dim_referee_raw_columns]
dim_referee_no_dup: pd.DataFrame = dim_referee_raw.drop_duplicates(subset=['referee']).reset_index(drop=True)
dim_referee_no_dup['referee_key'] = dim_referee_no_dup.index
dim_referee_columns: list = ['referee_key', 'referee']
dim_referee: pd.DataFrame = dim_referee_no_dup[dim_referee_columns]
print(dim_referee.shape)
dim_referee.head()

(234, 2)


,referee_key,referee
0,0,A. Taylor
1,1,A. Madley
2,2,P. Bankes
3,3,R. Jones
4,4,J. Gillett


In [40]:
dim_referee.to_sql(name='stg_referee', con=conn, index=False, if_exists='replace')

234

# dim_team (SCD 3)

In [43]:
dim_team_og_columns: list = ['teams.home.name', 'league.name', 'league.country', 'teams.home.logo']
dim_team_og_raw: pd.DataFrame = fixtures_raw[dim_team_og_columns]
dim_team_og_no_dup: pd.DataFrame = dim_team_og_raw.drop_duplicates(subset=['teams.home.name']).reset_index(drop=True)
dim_team_og_no_dup['team_key'] = dim_team_og_no_dup.index

dim_team_og: pd.DataFrame = dim_team_og_no_dup.rename(columns={
    'teams.home.name': 'team_name',
    'league.name': 'league_name',
    'league.country': 'league_country',
    'teams.home.logo': 'team_logo_url',
})
print(dim_team_og.shape)
dim_team_og.head()

(143, 5)


,team_name,league_name,league_country,team_logo_url,team_key
0,Crystal Palace,Premier League,England,https://media.api-sports.io/football/teams/52.png,0
1,Fulham,Premier League,England,https://media.api-sports.io/football/teams/36.png,1
2,Bournemouth,Premier League,England,https://media.api-sports.io/football/teams/35.png,2
3,Leeds,Premier League,England,https://media.api-sports.io/football/teams/63.png,3
4,Leicester,Premier League,England,https://media.api-sports.io/football/teams/46.png,4


In [44]:
dim_team_og.to_csv(path_or_buf='dim_team_og_new.csv', index=False)

In [50]:
dim_team_final: pd.DataFrame = pd.read_csv(filepath_or_buffer='dim_team_final_og_new.csv')
dim_team_final.head()

,team_key,team_name,league_name,league_country,team_logo_url,tm_api_club_id,2017_status,2018_status,2019_status,2020_status,2021_status,2022_status
0,0,Crystal Palace,Premier League,England,https://media.api-sports.io/football/teams/52.png,873.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Fulham,Premier League,England,https://media.api-sports.io/football/teams/36.png,931.0,NaN,Relegated,NaN,Relegated,NaN,NaN
2,2,Bournemouth,Premier League,England,https://media.api-sports.io/football/teams/35.png,989.0,NaN,NaN,Relegated,NaN,NaN,NaN
3,3,Leeds,Premier League,England,https://media.api-sports.io/football/teams/63.png,399.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Leicester,Premier League,England,https://media.api-sports.io/football/teams/46.png,1003.0,NaN,NaN,UEFA Europa League,UEFA Europa League,NaN,NaN


In [59]:
dim_team_final.to_sql(name='stg_team_final', con=engine, index=False, if_exists='replace')

143